In [3]:
import os, sys, re, json, time, unittest
import itertools, collections
import re
import operator
from sklearn import datasets, linear_model
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
#read in data to clean and separate into training and test set
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))


good = []
bad = []
score = []
tags = []
lines = []

#1 is good
#0 is not

#based on the distribution of scores, I rounded up the mean to 95 and used that to determine good and bad
#turns out the distribution between good and bad is pretty even.
for line in open('naomit_review_scores_final').readlines():
    data = line.split('\t')
    clean_score = re.sub("[^0-9]", "", data[1])
    desc = data[0]
    if clean_score != '' and int(clean_score) <= 100 and int(clean_score) > 0:
        if int(clean_score) >= 95:
            for each in desc.lower().split('.'):
                if each != ' ' and each != '':
                    good.append(re.sub('[^A-Za-z]', ' ', each))
                    #pre_lines.append((re.sub('[^A-Za-z]', ' ', each), 'good'))
                    lines.append(re.sub('[^A-Za-z]', ' ', each),)
                    score.append(int(clean_score))
                    tags.append(int(1))
           
        elif int(clean_score) < 95:
            for each in desc.lower().split('.'):
                if each != ' ' and each != '':
                    #pre_lines.append((re.sub('[^A-Za-z]', ' ', each), 'bad'))
                    lines.append(re.sub('[^A-Za-z]', ' ', each))
                    bad.append(re.sub('[^A-Za-z]', ' ', each))
                    score.append(int(clean_score))
                    tags.append(int(0))


In [5]:
print len(good)
print len(bad)

#lines = good + bad
print len(lines)
print len(score)
print len(tags)
for x in range(0,10):
    print good[x]
    print score[x]

1318902
861171
2180073
2180073
2180073
super clean and convenient to public transit
96
great location  very accessible to public transport  coffee shops  the mission and castro  the apartment was very comfortable and well equipped
96
 it was lovely having a garden courtyard and we even saw hummingbirds  however  it was difficult to reach our host to confirm arrangements and how to get in to the apartment
96
 we didn t receive a confirmation email or welcome info and compared to other airbnb stays it was sub standard client service
96
 it meant we were waiting on doorstep wondering if it was all going to be ok
96
 it was  but would have preferred not having to wonder
96
 the apartment was under the living area of the house and with two boisterous boys we were woken up early by running and thumping above us on what seemed rice paper ceilings
96
 despite that we enjoyed our time in san fransisco but would stay somewhere else next time
96
my husband and i had a great stay here for three ni

In [6]:
training = lines[:len(lines)/2]
testing = lines[len(lines)/2:]

tag_train = tags[:len(tags)/2]
tag_test = tags[len(tags)/2:]

print len(training)
print len(tag_train)
print lines[0]
print tags[0]

1090036
1090036
super clean and convenient to public transit
1


In [7]:
#removing stop words
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
no_stop_words = []
         
for each in lines:
    sentence = ""
    words = word_tokenize(each)
    for w in words:
        if w not in stop_words:
            sentence = sentence + " " + w

    no_stop_words.append(sentence)
    
print no_stop_words[1]

 great location accessible public transport coffee shops mission castro apartment comfortable well equipped


In [12]:
#LR with all words
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(lines)
y = tags
X_train_0 = X[:len(lines)/2]
X_test_0 = X[len(lines)/2:]
Y_train_0 = y[:len(tags)/2]
Y_test_0 = y[len(tags)/2:]

# Train model
clf_0 = LogisticRegression().fit(X_train_0, Y_train_0)
# Predict on training set
pred_y_0 = clf_0.predict(X_test_0)

In [13]:
#without taking out stop words
print( accuracy_score(pred_y_0, Y_test_0) )
print( np.unique( pred_y_0 ) )

0.635001380687
[0 1]


In [14]:
#LR without stop words
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(no_stop_words)
y = tags
X_train_1 = X[:len(no_stop_words)/2]
X_test_1 = X[len(no_stop_words)/2:]
Y_train_1 = y[:len(tags)/2]
Y_test_1 = y[len(tags)/2:]


# Train model
clf_1 = LogisticRegression().fit(X_train_1, Y_train_1)
# Predict on training set
pred_y_1 = clf_1.predict(X_test_1)

In [15]:
#after stop words
print( accuracy_score(pred_y_1, Y_test_1) )
print( np.unique( pred_y_1 ) )

0.632011573919
[0 1]


In [16]:
#######UPSAMPLING "BAD" CATEGORY TO MATCH GOOD#########

from sklearn.utils import resample

# Separate majority and minority classes
lines_majority = good
lines_minority = bad
 
# Upsample minority class
bad_upsampled = resample(lines_minority, 
                         replace=True,     # sample with replacement
                         n_samples=1318902,    # to match majority class
                         random_state=123) # reproducible results
 
    
# Combine majority class with upsampled minority class
print len(bad_upsampled)
print len(good)

1318902
1318902


In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from random import shuffle


new_list = []

for each in good:
    new_list.append((each, int(1)))

print len(new_list)
bad_upsampled = list(bad_upsampled)
for each in bad_upsampled:
    new_list.append((each, int(0)))

print len(new_list)



shuffle(new_list)

new_lines = []
new_tags = []

for each in new_list:
    new_lines.append(each[0])
    new_tags.append(each[1])
                     
                     
print len(new_lines)
print len(new_tags)

1318902
2637804
2637804
2637804


In [18]:
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(new_lines)
y = new_tags

X_train_2 = X[:len(new_lines)/2]
X_test_2 = X[len(new_lines)/2:]
Y_train_2 = new_tags[:len(new_tags)/2]
Y_test_2 = new_tags[len(new_tags)/2:]


# Train model
clf_2 = LogisticRegression().fit(X_train_2, Y_train_2)
 
# Predict on training set
pred_y_2 = clf_2.predict(X_test_2)

In [19]:
print( accuracy_score(pred_y_2, Y_test_2) )
print( np.unique( pred_y_2 ) )

0.651537415214
[0 1]
